In [1]:
!pip install bs4
!pip install openpyxl
!pip install fake_useragent
!pip install loguru
!pip install pymorphy2
!pip install flair
!pip install transformers sentencepiece
!pip install sacremoses
!pip install jq
!pip install slovnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=050dea36a01cd9f54ab1c3596c0222f82c9dfb0db5e3d6f481282731816d1b22
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.0 MB/s e

In [2]:
#  Импорт библиотек
import aiohttp
import asyncio
import pandas as pd
import numpy as np
import pickle
import pymorphy2
import re
import requests
from urllib.parse import quote_plus
from bs4 import BeautifulSoup as bs
from datetime import date, timedelta, datetime
from fake_useragent import UserAgent
from loguru import logger
from abc import ABC, abstractmethod
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import random_projection
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_distances

import os
import gc
import nltk
from flair.data import Sentence
from flair.nn import Classifier
from torch.utils.data import DataLoader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import OrderedDict
from transformers import MPNetPreTrainedModel, MPNetModel, AutoTokenizer
import torch
import math
from navec import Navec
from slovnet import NER

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# ИНН

In [4]:
# Функция для определения имени компании по ИНН
def inn_to_company(inn: str):
  url_inn = f'https://egrul.itsoft.ru/{inn}.json'
  inn_request = requests.get(url_inn)
  inn_json = inn_request.json()
  full_company_name = inn_json['СвЮЛ']['СвНаимЮЛ']['@attributes']['НаимЮЛПолн']
  company_name = re.search(r'\".+\"', full_company_name)
  return company_name.group(0).replace('"', '').lower()

# Парсеры

In [5]:
# Интерфейс для всех парсеров
class IParser(ABC):
  def __init__(self):
    self.data = pd.DataFrame()

  @abstractmethod
  async def _get_header(self):
    """ Метод для получения header """

  @abstractmethod
  async def _get_url(self):
    """ Метод для получения url """

  @abstractmethod
  async def _get_data(self, size: int):
    """ Метод для получения urls """

  async def _get_data_titles(self, url:str, title:str, text:str, date:str):
    return {
          url: 'url',
          title: 'title',
          text: 'text',
          date: 'date'
          }

  async def _fill_data(self, frames, data_titles):
    data = pd.concat(frames)
    data.rename(columns = data_titles, inplace = True)
    data = data.drop_duplicates(subset=['url'])
    data['publication_date'] = pd.to_datetime(data['date'], unit='s')
    data['company'] = company
    data['inn'] = inn
    self.data = data[['url', 'title', 'text', 'publication_date', 'inn','company']].dropna(axis=0, how='any')

  async def start(self):
    logger.info(f'{self.__class__.__name__} start')
    await self._get_data()
    logger.info(f'{self.__class__.__name__} end')

In [6]:
# Парсер Лента
class Lenta(IParser):
    async def _get_header(self):
      pass

    async def _get_url(self):
      url = 'https://lenta.ru/search/v2/process?'\
          + 'query={}&'\
          + 'from={}&'\
          + 'size={}&'\
          + 'sort=2&'\
          + 'title_only=0&'\
          + 'domain=1&'\
          + 'modified%2Cformat=yyyy-MM-dd&'\
          + 'modified%2Cfrom={}&'\
          + 'modified%2Cto={}'\

      return url

    async def _get_data(self, size: int = 500):
      page = 0
      frames = []
      url = await self._get_url()

      async with aiohttp.ClientSession() as session:
        while True:
          current_url = url.format(company,page, size, date_start.strftime("%Y-%m-%d"), date_end.strftime("%Y-%m-%d"))
          async with session.get(current_url) as response:
            if response.status != 200:
              logger.error(f'{self.__class__.__name__} status code {response.status}, url = {current_url}')
              await asyncio.sleep(10)
              continue
            data = await response.json()

          if len(data['matches']) == 0:
            break

          frames.append(pd.DataFrame(data['matches']))
          page += size

      try:
        data_titles = await self._get_data_titles('url', 'title', 'text', 'pubdate')
        await self._fill_data(frames, data_titles)
      except:
        logger.info(f'{self.__class__.__name__}: нет новостей')
        pass

In [7]:
# Парсер Rbc
class Rbc(IParser):
    async def _get_header(self):
      pass

    async def _get_url(self):
      url = 'https://www.rbc.ru/search/ajax/?' +\
        'query={}&' +\
        'dateFrom={}&' +\
        'dateTo={}&' +\
        'page={}'

      return url

    async def _get_data(self, size: int = 1):
      page = 0
      date_from = date_start
      date_to = date_end
      frames = []
      url = await self._get_url()

      async with aiohttp.ClientSession() as session:
        while True:
          current_url = url.format(company, date_from.strftime('%d.%m.%Y'), date_to.strftime('%d.%m.%Y'), page)
          async with session.get(current_url) as response:
            data = await response.json()
            if response.status != 200:
              logger.error(f'{self.__class__.__name__} status code {response.status}, url = {current_url}')
              await asyncio.sleep(10)
              continue

          if not data['moreExists']:
            break

          search_table = pd.DataFrame(data['items'])

          text = []
          for v in search_table['fronturl']:
              text.append(await self._get_article_data(v, session))

          search_table['text'] = text

          frames.append(search_table)

          page += size
          logger.info(f"{page}")
          if page == 100:
            date_to = datetime.strptime(search_table['publish_date'].iat[-1][:10], '%Y-%m-%d').date()
            page = 0
            logger.info(f'{self.__class__.__name__}: новые даты {date_from} - {date_to}')

      try:
        data_titles = await self._get_data_titles('fronturl', 'title', 'text', 'publish_date_t')
        await self._fill_data(frames, data_titles)
      except:
        logger.info(f'{self.__class__.__name__}: нет новостей')
        pass


    async def _get_article_data(self, url: str, session):
        """
        Возвращает описание и текст статьи по ссылке
        """
        async with session.get(url) as r:
          soup = bs(await r.text(), features="lxml") # features="lxml" чтобы не было warning
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None

        return text

In [8]:
# Парсер Регнум
class Regnum(IParser):
    async def _get_header(self):
      pass

    async def _get_url(self):
      url = 'https://regnum.ru/api/search/materials?' +\
        'page={}&' +\
        'searchText={}&' +\
        'typeIds=0&' +\
        'typeIds=1&' +\
        'typeIds=2&' +\
        'typeIds=3&' +\
        'typeIds=6&' +\
        'dateFrom={}&' +\
        'dateTo={}&' +\
        'order=desc'

      return url

    async def _get_data(self, size: int = 1):
        page = 0
        frames = []
        url = await self._get_url()

        async with aiohttp.ClientSession() as session:
          while True:
            current_url = url.format(page, company, date_start.strftime("%d-%m-%Y"), date_end.strftime("%d-%m-%Y"))
            async with session.get(current_url) as response:
              if response.status != 200:
                logger.error(f'{self.__class__.__name__} status code {response.status}, url = {current_url}')
                await asyncio.sleep(10)
                continue
              data = await response.json()

            if data['pagination']['next'] == 0:
              break

            search_table = pd.DataFrame(data['hits'])

            text = []
            for v in search_table['url']:
              text.append(await self._get_article_data(v, session))

            search_table['text'] = text

            frames.append(search_table)
            page += size

        try:
          data_titles = await self._get_data_titles('url', 'header', 'text', 'date')
          await self._fill_data(frames, data_titles)
        except:
          logger.info(f'{self.__class__.__name__}: нет новостей')
          pass


    async def _get_article_data(self, url: str, session):
        """
        Возвращает описание и текст статьи по ссылке
        """
        async with session.get(f'https://regnum.ru{url}') as r:
          soup = bs(await r.text(), features="lxml") # features="lxml" чтобы не было warning
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None
        return text

# Удаление мусора

In [9]:
class RepeatRemover():
    morph = pymorphy2.MorphAnalyzer()
    morph_dct = {}

    def preprocess_text(self, text):
        text = text.lower()
        text = text.replace('\r', ' ').replace('\n', ' ')
        text  = self.only_valid_symb(text)
        norm_text = self.only_valid_forms(text)
        return norm_text


    def only_valid_symb(self, text: str)-> str:
        valid = set('йцукенгшщзхъфывапролджэячсмитьбюё ')
        txt =  ''.join(x for x in text if x in valid)
        while '  ' in txt:
            txt = txt.replace('  ', ' ')
        return txt.strip()

    def only_valid_forms(self, text)->str:
        filter_forms = {'ADJF', 'ADJS', 'COMP', 'NUMR', 'NPRO', 'PREP', 'CONJ', 'PRCL', 'INTJ'}
        lst = []
        for word in text.split():
            tag = self.morph_parse(word)
            if any([x in tag.tag for x in filter_forms]):
                continue
            lst.append(tag.normal_form)
        return ' '.join(lst)

    def morph_parse(self, word:str)->str:
        if word not in self.morph_dct:
            tag = self.morph.parse(word)[0]
            self.morph_dct[word] = tag
        return self.morph_dct[word]

    def start(self, articles):
        data = articles.to_dict('records')
        _text = [self.preprocess_text(x['text']) for x in data if type(x['text']) is str and x['text']!='']
        vectorized = TfidfVectorizer(min_df=2).fit_transform(_text)

        __X = np.ceil(vectorized.toarray())
        dist = pairwise_distances(__X, metric='cosine')
        np.fill_diagonal(dist, 1)
        pairs = set()
        for x, y in np.argwhere(dist < 0.15):
            pairs.add(tuple(sorted([x, y])))

        clones = set()
        for pair in pairs:
            copy_1 = data[pair[0]]
            copy_2 = data[pair[1]]
            if abs((copy_1['publication_date'] - copy_2['publication_date']).days) < 7:
                clones.add(pair[1])

        data_array = np.array(data)
        data_array_unique = np.delete(data_array, list(clones))
        data_unique = data_array_unique.tolist()
        return pd.DataFrame(data_unique)


# Проверка вхождений

In [10]:
async def is_company_in_text(text ,pattern):
  return bool(re.search(pattern,text))

async def get_occurrences_df(df, pattern):
  df['is_company_in_text'] = await asyncio.gather(*(is_company_in_text(v, pattern) for v in df['text']))
  return df[df['is_company_in_text']]

# LLM

In [58]:
def gpt_response(question):
    url = "http://5.39.220.103:5009/ask"

    data = {
        "messages": [
            {"role": "system", "content":
                                       f"Пиши только одно слово - да или нет."
                                       },
            {"role": "user", "content": f"{question}"}
        ]
    }

    response = requests.post(url, json=data)

    if response.status_code == 200:
        response_data = response.json()
        return response_data['response']
    else:
        return f"Error: {response.status_code}, {response.text}"

In [49]:
question = f"относиться ли данный текст к компании ромашка? Текст: Нецелевое использование участка карается штрафом в размере 0,5-1 процента от его кадастровой стоимости, но суммой не менее 10 тысяч рублей. Получить штраф можно также за мойку машины за забором (3-5 тысяч рублей) и складирование мусора (10 тысяч рублей). При повреждении чужого имущества сумма достигнет 50 тысяч рублей, не говоря о компенсации ущерба. Также штраф можно получить за заросли сорняков — если на участке расплодился борщевик Сосновского, разрослась пастушья сумка либо организованы посадки мяты, ромашки и амброзии."
answer = gpt_response(question, 'Ромашка', ttt)
answer

'да'

In [13]:
async def gpt_response_async(question, company, article):
    url = "http://5.39.220.103:5009/ask"

    data = {
        "messages": [
            {"role": "system", "content": f"Ты определяешь, связана статья с компанией {company}"
                                       f"Тебе поступают статья и название компании, по которым ты должен определить, говорится ли именно о компании {company}."
                                       f"В статье должно говориться именно о компании {company}. {company} должна быть именно компанией и ничем другим."
                                       f"Компания это юридическое лицо, созданное одним или несколькими физическими лицами для ведения бизнеса"
                                       f"Ты определяешь, относятся ли действия, деятельность компании, описанные в статье, к компании {company}. Если относятся - отвечаешь да, если нет - отвечаешь нет."
                                       f"Отвечай только на поставленный вопрос, не уходи от темы вопроса. "
                                       f"Пиши только одно слово - да или нет."
                                       f"Статья: \n {article}"
                                       f"Название компании: \n {company}."
                                       },
            {"role": "user", "content": f"{question}"}
        ]
    }

    async with aiohttp.ClientSession() as session:
      response = await session.post(url, json=data)

    if response.status == 200:
        response_data = await response.json()
        return response_data['response']
    else:
        return f"Error: {response.status_code}, {response.text}"

# Классификация с помощью ESGify

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Definition of ESGify class because of custom,sentence-transformers like, mean pooling function and classifier head
class ESGify(MPNetPreTrainedModel):
    """Model for Classification ESG risks from text."""

    def __init__(self,config): #tuning only the head
        """
        """
        super().__init__(config)
        # Instantiate Parts of model
        self.mpnet = MPNetModel(config,add_pooling_layer=False)
        self.id2label =  config.id2label
        self.label2id =  config.label2id
        self.classifier = torch.nn.Sequential(OrderedDict([('norm',torch.nn.BatchNorm1d(768)),
                                                ('linear',torch.nn.Linear(768,512)),
                                                ('act',torch.nn.ReLU()),
                                                ('batch_n',torch.nn.BatchNorm1d(512)),
                                                ('drop_class', torch.nn.Dropout(0.2)),
                                                ('class_l',torch.nn.Linear(512 ,47))]))


    def forward(self, input_ids, attention_mask):
         # Feed input to mpnet model
        outputs = self.mpnet(input_ids=input_ids,
                             attention_mask=attention_mask)

        # mean pooling dataset and eed input to classifier to compute logits
        logits = self.classifier( mean_pooling(outputs['last_hidden_state'],attention_mask))

        # apply sigmoid
        logits  = 1.0 / (1.0 + torch.exp(-logits))
        return logits

In [ ]:
def get_texts_with_masks(texts):
    navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
    ner = NER.load('slovnet_ner_news_v1.tar')
    ner.navec(navec)

    stop_words = set(stopwords.words('russian'))
    tag_list = ['FAC','LOC','ORG','PER']
    texts_with_masks = []
    for example_sent in texts:
        filtered_sentence = []
        word_tokens = word_tokenize(example_sent)
        # converts the words in word_tokens to lower case and then checks whether
        #they are present in stop_words or not
        for w in word_tokens:
            if w.lower() not in stop_words:
                filtered_sentence.append(w)
        # make a sentence
        sent = ' '.join(filtered_sentence)
        markup = ner(sent)
        k = 0
        new_string = ''
        start_t = 0
        for span in markup.spans:
              if span.type in tag_list:
                if span.start>start_t :
                    new_string+=sent[start_t:span.start]
                start_t = span.stop
                new_string+= f'<{span.type}>'
        new_string+=sent[start_t:-1]
        texts_with_masks.append(new_string)
    return texts_with_masks

In [ ]:
def get_classes(df, texts_with_masks, model, tokenizer):
    weights = pd.read_excel('trashs_esgify_ru.xlsx')
    weights_dict = {}
    for row in weights.index:
        weights_dict[weights['Unnamed: 0'][row]] =  weights['trash'][row]

    for e in range(0, len(df), 10):
        last_elem = e + 10
        if e + 10 > len(df):
            last_elem = len(df)
        text_slice = texts_with_masks[e:last_elem]
        to_model = tokenizer.batch_encode_plus(
                          text_slice,
                          add_special_tokens=True,
                          max_length=512,
                          return_token_type_ids=False,
                          padding="max_length",
                          truncation=True,
                          return_attention_mask=True,
                          return_tensors='pt',
                    )
        to_model.to(device)
        results = model(**to_model)
        results.to(device)
        # for i in range(len(results)):
        #     print('-------------')
        #     for j in torch.topk(results, k=10).indices.tolist()[i]:
        #         print(f"{model.id2label[j]}: {np.round(results[i][j].item(), 3)}")
        for i in range(len(df[e:last_elem])):
            classes = ''
            for j in range(47):
                label = model.id2label[j]
                if results[i][j].item() >= weights_dict[label]:
                    classes += f'{label};'
            if len(classes) == 0:
                classes = 'Not Relevant to ESG;'
            classes = classes[:len(classes) - 1]
            df.loc[e + i, "classes"] = classes
        gc.collect()
        torch.cuda.empty_cache()
    return df

In [ ]:
def classify(df):
    model = ESGify.from_pretrained('ai-lab/ESGify')
    model.to(device)
    tokenizer = AutoTokenizer.from_pretrained('ai-lab/ESGify')

    for i in range(len(df)):
        if type(df['text'][i]) == float or not bool(df['text'][i].replace(' ', '')):
            df.drop(i, inplace=True)
            continue
        if len(df['text'][i]) > 5000:
            df.loc[i, "text"] = df['text'][i][:5000]
    ru_texts = list(df['text'])
    texts_with_masks = get_texts_with_masks(ru_texts)
    df['classes'] = ''
    df = get_classes(df, ru_texts, model, tokenizer)
    df = df.loc[df['classes'] != "Not Relevant to ESG"]
    return df

# Начало работы

In [89]:
# Настройка
inn = '7736050003'
date_start = date(year=2023, month=1, day=1)
date_end = date.today()

In [90]:
# Получаем название компании по ИНН
company = inn_to_company(inn)
company

'газпром'

In [91]:
# Паттерн для поиска вхождений компании в тексте
company_pattern = f'{company[:-1]}\w'+'{,2}'
pattern = re.compile(f"(?i){company_pattern}")
pattern

re.compile(r'(?i)газпро\w{,2}', re.IGNORECASE|re.UNICODE)

In [92]:
# Данные для работы с header (пока не используется)
query = quote_plus(company)
ua = UserAgent()

In [93]:
# Это для ассинхронного запуска
parsers = [Rbc(),Lenta(),Regnum()]

async def main():
  tasks = []
  for parser in parsers:
    tasks.append(parser.start())

  await asyncio.gather(*tasks)

In [94]:
# Запускаем асинхронную работу парсеров
await main()

2024-10-27 05:12:40.403 | INFO     | __main__:start:36 - Rbc start
2024-10-27 05:12:40.407 | INFO     | __main__:start:36 - Lenta start
2024-10-27 05:12:40.409 | INFO     | __main__:start:36 - Regnum start
2024-10-27 05:12:47.559 | INFO     | __main__:start:38 - Lenta end
2024-10-27 05:12:55.366 | INFO     | __main__:_get_data:46 - 1
2024-10-27 05:13:08.174 | INFO     | __main__:_get_data:46 - 2
2024-10-27 05:13:24.449 | INFO     | __main__:_get_data:46 - 3
2024-10-27 05:13:37.558 | INFO     | __main__:_get_data:46 - 4
2024-10-27 05:13:50.095 | INFO     | __main__:_get_data:46 - 5
2024-10-27 05:14:03.857 | INFO     | __main__:_get_data:46 - 6
2024-10-27 05:14:14.655 | INFO     | __main__:_get_data:46 - 7
2024-10-27 05:14:26.985 | INFO     | __main__:_get_data:46 - 8
2024-10-27 05:14:39.066 | INFO     | __main__:_get_data:46 - 9
2024-10-27 05:14:50.970 | INFO     | __main__:_get_data:46 - 10
2024-10-27 05:15:03.186 | INFO     | __main__:_get_data:46 - 11
2024-10-27 05:15:15.596 | INFO  

In [95]:
# Смотрим кол-во статей для каждого парсера
for parser in parsers:
  print(f'{parser.__class__.__name__}: кол-во статей {len(parser.data)}')

Rbc: кол-во статей 2413
Lenta: кол-во статей 1144
Regnum: кол-во статей 599


In [96]:
parsers[0].data.head()

,url,title,text,publication_date,inn,company
0,https://www.rbc.ru/spb_sz/26/10/2024/67175c1c9...,Петербургский суд принял решение по иску на ₽1...,Суд в Санкт-Петербурге частично удовлетворил и...,2024-10-26 06:34:41,7736050003,газпром
1,https://www.rbc.ru/quote/news/article/671a0a7e...,Ренкинг РБК 100: 5 главных интриг в акциях рос...,РБК представил ренкинг 100 компаний по объему ...,2024-10-26 05:30:18,7736050003,газпром
2,https://www.rbc.ru/sport/26/10/2024/671b6e9d9a...,Сменится ли лидер чемпионата России по футболу...,Футболисты московского ЦСКА в прошлом туре об...,2024-10-25 21:00:22,7736050003,газпром
3,https://companies.rbc.ru/news/MBeaouk7bB/sibst...,«Сибстекло» одержало победу в конкурсе «Дело в...,"Проект «Круговорот стекла в заводе», представл...",2024-10-25 08:57:19,7736050003,газпром
4,https://nn.plus.rbc.ru/partners/671b75227a8aa9...,Как нижегородец попал в лидеры России по автом...,"Получайте рассылку с новостями, которые касают...",2024-10-25 07:38:26,7736050003,газпром


In [97]:
# Тут начало удаления повторок
df = pd.concat([parser.data for parser in parsers], ignore_index=True)
print(f'перваночальное кол-во статей: {len(df)}')

перваночальное кол-во статей: 4156


In [98]:
df.isna().sum()

,0
url,0
title,0
text,0
publication_date,0
inn,0
company,0


In [ ]:
remover = RepeatRemover()
df_clear = remover.start(df)
print(f'кол-во статей после удаления повторок: {len(df_clear)}') #10 сек 422 статьи

In [75]:
df_clear.head()

,url,title,text,publication_date,inn,company
0,https://www.rbc.ru/wine/news/6718b1f19a7947d25...,Рецепты настоек в домашних условиях и правила ...,"Настойки — категория напитков, которая в совре...",2024-10-24 07:25:09,1708003392,ромашка
1,https://pro.rbc.ru/demo/66e99e129a79474635092ed6,Так ли безопасен безалкогольный алкоголь,Всплеск популярности безалкогольного алкоголя ...,2024-09-25 12:28:34,1708003392,ромашка
2,https://trends.rbc.ru/trends/social/66f155309a...,"Инсомния, гиперсомния и парасомния: что такое ...","Получайте рассылку с новостями, которые касают...",2024-09-23 13:40:11,1708003392,ромашка
3,https://style.rbc.ru/beauty/66e412f59a7947423c...,"Для тех, кто искал: новые средства для защиты ...",Бренд Valmont дополнил линию LumiCity солнцеза...,2024-09-13 14:56:51,1708003392,ромашка
4,https://www.rbc.ru/wine/news/66e0199c9a7947b5e...,Врачи назвали лучшие напитки для Дня трезвости,Врачи назвали лучшие напитки для дня трезвости...,2024-09-10 11:24:39,1708003392,ромашка


In [76]:
df_occurrences = await get_occurrences_df(df_clear, pattern)
print(f'кол-во статей после проверки вхождения: {len(df_occurrences)}')

кол-во статей после проверки вхождения: 340


In [77]:
df_occurrences.head()

,url,title,text,publication_date,inn,company,is_company_in_text
0,https://www.rbc.ru/wine/news/6718b1f19a7947d25...,Рецепты настоек в домашних условиях и правила ...,"Настойки — категория напитков, которая в совре...",2024-10-24 07:25:09,1708003392,ромашка,True
3,https://style.rbc.ru/beauty/66e412f59a7947423c...,"Для тех, кто искал: новые средства для защиты ...",Бренд Valmont дополнил линию LumiCity солнцеза...,2024-09-13 14:56:51,1708003392,ромашка,True
4,https://www.rbc.ru/wine/news/66e0199c9a7947b5e...,Врачи назвали лучшие напитки для Дня трезвости,Врачи назвали лучшие напитки для дня трезвости...,2024-09-10 11:24:39,1708003392,ромашка,True
5,https://style.rbc.ru/impressions/66dee70a9a794...,Стартовал О2 — первый всероссийский Фестиваль ...,"Фестивальное меню, «Бор» Гастрономическая одис...",2024-09-09 12:57:25,1708003392,ромашка,True
6,https://kavkaz.rbc.ru/kavkaz/06/09/2024/66dae6...,Аграрии юга намерены заработать на подсолнечни...,В южных регионах началась уборка масличных кул...,2024-09-06 11:23:57,1708003392,ромашка,True


In [82]:
morph = pymorphy2.MorphAnalyzer()

df = df_occurrences.reset_index()
for i in range(len(df)):
            # print(df['text'][i])
            if type(df['text'][i]) == float:
                df.drop(i, inplace=True)
                continue

            # if company[:(len(company)//2)] not in df['text'][i]: # если компания не содержится в статье, удаляем статью
            #     df.drop(i, inplace=True)
            #     continue

            # summary = re.findall(r"[\w']+|[.,!?;]", text_summarizer(df['text'][i])) # рефератор текста + проверка вхождения слова в название компании
            # isCompany = False
            # for j in range(len(summary)):
            #     word_normal_form = morph.parse(summary[j])[0].normal_form.lower()
            #     if len(word_normal_form) > len(company) // 2 and word_normal_form in company.lower():
            #       isCompany = True
            #       break
            # if not isCompany:
            #     df.drop(i, inplace=True)
            #     continue

            # isCompany = False # проверка вхождения слова в название компании
            # text = re.findall(r"[\w']+|[.,!?;]", df['text'][i])
            # for j in range(len(text)):
            #     word_normal_form = morph.parse(text[j])[0].normal_form.lower()
            #     if len(word_normal_form) > len(company) // 2 and word_normal_form in company.lower():
            #       isCompany = True
            #       break
            # if not isCompany:
            #     df.drop(i, inplace=True)
            #     continue

            # summary = text_summarizer(df['text'][i])# рефератор текста (сокращение, выделение основной мысли)
            # print(summary)
            # if company[:4] not in summary: # если компания не содержится в рефераторе, удаляем статью
            #   df.drop(i, inplace=True)
            #   continue

            text = df['text'][i]

            if len(text) > 12000:
              text = text[:12000]
            question = f"относиться ли данный текст к юридическому лицу {company}? Текст: {text}"
            answer = gpt_response(question).lower()
            if "да" not in answer:
              df.drop(i, inplace=True)
              continue

            # if check_company_in_article(df['text'][i], i, company, df): # проверка на совпадение заданной компании с самой часто упоминающейся в статье
            #   df.drop(i, inplace=True)
            #   continue
print(len(df)) #21мин25сек - 340

2


In [83]:
df.head()

,index,url,title,text,publication_date,inn,company,is_company_in_text
85,118,https://pro.rbc.ru/demo/63f4294b9a794730ebb408a0,Как компании сэкономить на платежном оборудова...,"Для того чтобы принимать платежи по картам, ну...",2023-03-02 09:37:01,1708003392,ромашка,True
254,325,https://lenta.ru/news/2021/10/19/izbili/,Россиян избили в магазине из-за просроченной н...,Фото: Алексей Даничев / РИА Новости Анна Филип...,2021-10-19 16:27:23,1708003392,ромашка,True


In [81]:
df.to_csv(f'without_class_{inn}_{company}.csv')

Далее для корректной работы нужный файлы: trashs_esgify_ru.xlsx, slovnet_ner_news_v1.tar и navec_news_v1_1B_250K_300d_100q.tar

In [ ]:
# Начало классфикации
df_with_classes = classify(df.reset_index())

In [ ]:
len(df_with_classes)

78

In [ ]:
df_with_classes.head()

,level_0,index,url,title,text,publication_date,inn,company,is_company_in_text,classes
0,0,0,https://www.rbc.ru/economics/18/09/2024/66e96d...,Верховный суд указал на возможность взимать «Н...,Судебная коллегия по экономическим спорам Верх...,2024-09-18 06:30:12,5035029183,спецстрой,True,Soil and Groundwater Impact
1,1,1,https://t.rbc.ru/tyumen/15/05/2024/6644a9f99a7...,Тюменские власти сэкономят на ремонте областно...,"Тюменские власти определились с подрядчиком, к...",2024-05-15 12:55:34,5035029183,спецстрой,True,Soil and Groundwater Impact
2,2,2,https://www.rbc.ru/politics/14/04/2024/661ae05...,Житель Орска рассказал о последствиях наводнен...,"Житель Орска Сергей, рассказывая в эфире прог...",2024-04-14 04:58:46,5035029183,спецстрой,True,Soil and Groundwater Impact
3,3,3,https://www.rbc.ru/society/12/04/2024/6618fd4f...,В поселке Газодобытчиков в Оренбургском районе...,В поселке Газодобытчиков в связи с риском прор...,2024-04-12 09:34:58,5035029183,спецстрой,True,Environmental Management;Soil and Groundwater ...
4,4,4,https://www.rbc.ru/society/12/04/2024/6618f318...,МЧС сообщило о восстановлении участка дамбы в ...,"В Орске восстановили участок дамбы, который пр...",2024-04-12 08:55:58,5035029183,спецстрой,True,Soil and Groundwater Impact


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/data_table.py", line 192, in _repr_mimebundle_
    dataframe = self._preprocess_dataframe()
  File "/usr/local/lib/python3.10/dist-packages/google/colab/data_table.py", line 180, in _preprocess_dataframe
    dataframe = dataframe.reset_index()
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py", line 6472, in reset_index
    new_obj.insert(
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py", line 5158, in insert
    raise ValueError(f"cannot insert {column}, already exists")
ValueError: cannot insert level_0, already exists
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/data_table.py", line 204, in _repr_javascript_module_
    return self._gen_js(self._preprocess_dataframe())
  File "/usr/local/lib/python3.10/dist-packages/google/colab/data_table.py", line 180, in _preprocess_dataframe
    dataframe = dataf

In [ ]:
filtered_df = df_with_classes[df_with_classes['classes'] == 'Not Relevant to ESG']

In [ ]:
filtered_df

,level_0,index,url,title,text,publication_date,inn,company,is_company_in_text,classes


In [ ]:
df_with_classes.to_csv(f'with_classes_{company}3.csv')